In [1]:
import google.generativeai as genai
import json
import re
from geopy.distance import geodesic

# 모델 생성
model = genai.GenerativeModel('gemini-1.5-flash')

def parse_days(input_str):
    match = re.search(r"\d+", input_str)
    return int(match.group()) if match else 1

# 사용자 입력 받기
print("여행 일정 추천을 위해 아래 항목들을 입력해주세요:")
location = input("여행지 위치 (예: 제주도, 유럽): ")
days = parse_days(input("여행 기간 (예: 2일, 3): "))

# 여행 테마 (사용자가 입력)
theme = input("여행 테마 (예: 맛집, 카페, 박물관 등): ")

# 최대 거리 입력 받기 (단위: km, 기본값은 제한 없음으로 설정)
max_distance_input = input("최대 거리 (km, 예: 5, 생략 시 거리 제한 없음으로 설정): ")

# 거리 제한 없음으로 설정, 입력이 없으면 제한이 없는 것으로 처리
max_distance = float(max_distance_input) if max_distance_input else None
min_distance = None  # 기본적으로 거리 제한 없음

# AI에 메시지 전송
query = f"""
당신은 여행 일정 생성 AI입니다.
사용자의 요청에 맞춰 {location}에서 {days}일 동안 '{theme}' 테마 여행 일정을 생성하세요.
그리고 추천되는 장소들은 '{theme}' 테마에 맞는 장소를 추천해 주세요.

응답은 아래 JSON 형식을 따르세요:
{{
    "itinerary": [
        {{
            "day": 1,
            "places": [
                {{
                    "name": "장소 이름",
                    "type": "{theme}",
                    "location": "장소의 주소 또는 좌표"
                }},
                {{
                    "name": "장소 이름",
                    "type": "{theme}",
                    "location": "장소의 주소 또는 좌표"
                }},
                {{
                    "name": "장소 이름",
                    "type": "{theme}",
                    "location": "장소의 주소 또는 좌표"
                }}
            ]
        }} ,
        ... (각 날짜별 반복)
    ]
}}
"""

response = model.generate_content(query)

# 응답 처리
try:
    response_data = json.loads(response.text)
    print(json.dumps(response_data, indent=4, ensure_ascii=False))

    # 추천된 장소들의 좌표를 바탕으로 거리 계산
    valid_places = []
    start_coords = (37.4979, 127.0276)  # 예시 출발지 좌표 (서울 강남역)

    for day in response_data["itinerary"]:
        for place in day["places"]:
            place_name = place["name"]
            place_coords = place["location"]  # 실제 좌표로 변환 필요 (예시에서는 문자열로 입력됨)

            # 좌표가 문자열인 경우, 이를 처리하여 위도, 경도 값을 받아오는 코드 필요
            # 예: "37.5665, 126.9780" -> (37.5665, 126.9780)

            # 임시로 예시 좌표 입력 (실제 구현시 AI로부터 받은 실제 좌표 사용)
            place_coords = (37.5665, 126.9780)  # 예시 좌표 (서울 시청)

            # 거리 계산
            if min_distance is not None and max_distance is not None:
                distance = geodesic(start_coords, place_coords).km
                if min_distance <= distance <= max_distance:
                    valid_places.append({
                        "name": place_name,
                        "distance": distance,
                        "coords": place_coords
                    })
            else:
                valid_places.append({
                    "name": place_name,
                    "coords": place_coords
                })

    # 필터링된 장소 출력 (한 번에 3개 추천)
    if valid_places:
        print("추천된 장소 (거리 조건을 충족하는):")
        for i in range(min(3, len(valid_places))):
            place = valid_places[i]
            print(f"- {place['name']} (좌표: {place['coords']})")
    else:
        print(f"⚠️ 적합한 장소가 없습니다.")
except json.JSONDecodeError:
    print("⚠️ 응답이 JSON 형식이 아닙니다. 원본 출력:")
    print(response.text)


여행 일정 추천을 위해 아래 항목들을 입력해주세요:
⚠️ 응답이 JSON 형식이 아닙니다. 원본 출력:
```json
{
  "itinerary": [
    {
      "day": 1,
      "places": [
        {
          "name": "밀면 맛집 - 개금밀면",
          "type": "맛집",
          "location": "부산 부산진구 개금동 191-1"
        },
        {
          "name": "돼지국밥 맛집 - 신평돼지국밥",
          "type": "맛집",
          "location": "부산 사하구 장림동 241-1"
        },
        {
          "name": "해산물 맛집 - 자갈치시장",
          "type": "맛집",
          "location": "부산 중구 남포동5가 41-1"
        }
      ]
    },
    {
      "day": 2,
      "places": [
        {
          "name": "어묵 맛집 - 삼진어묵",
          "type": "맛집",
          "location": "부산 중구 중앙동 4가 16-1"
        },
        {
          "name": "단팥빵 맛집 - 옵스",
          "type": "맛집",
          "location": "부산 중구 광복동1가 28-1"
        },
        {
          "name": "카페 맛집 - 흰여울문화마을 카페 ( 여러 카페 중 선택 가능)",
          "type": "맛집",
          "location": "부산 영도구 영선동2가 흰여울길" 
        }
      ]
    }
  ]
}
```

